In [3]:
#utf-8
import hashlib
import io
import json
import os
import contextlib2
import numpy as np
import PIL.Image
import random
import sys
sys.path.append('/Users/hywel/Documents/GitSrc/models/research')

from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

import tensorflow as tf

In [4]:
a = "中国"
a.encode('utf-8')

b'\xe4\xb8\xad\xe5\x9b\xbd'

In [5]:
name = r"\u94c1\u58f3\u6253\u706b\u673a"
print(name.encode('ascii').decode('unicode_escape'))

铁壳打火机


In [6]:
def create_tf_example(image,
                      annotations_list,
                      image_dir,
                      category_index,
                      include_masks=False):
    """Converts image and annotations to a tf.Example proto.

    Args:
      image: dict with keys:
        [u'license', u'file_name', u'coco_url', u'height', u'width',
        u'date_captured', u'flickr_url', u'id']
      annotations_list:
        list of dicts with keys:
        [u'segmentation', u'area', u'iscrowd', u'image_id',
        u'bbox', u'category_id', u'id']
        Notice that bounding box coordinates in the official COCO dataset are
        given as [x, y, width, height] tuples using absolute coordinates where
        x, y represent the top-left (0-indexed) corner.  This function converts
        to the format expected by the Tensorflow Object Detection API (which is
        which is [ymin, xmin, ymax, xmax] with coordinates normalized relative
        to image size).
      image_dir: directory containing the image files.
      category_index: a dict containing COCO category information keyed
        by the 'id' field of each category.  See the
        label_map_util.create_category_index function.
      include_masks: Whether to include instance segmentations masks
        (PNG encoded) in the result. default: False.
    Returns:
      example: The converted tf.Example
      num_annotations_skipped: Number of (invalid) annotations that were ignored.

    Raises:
      ValueError: if the image pointed to by data['filename'] is not a valid JPEG
    """
    image_height = image['height']
    image_width = image['width']
    filename = image['file_name']
    image_id = image['id']

    full_path = os.path.join(image_dir, filename)
    with tf.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    key = hashlib.sha256(encoded_jpg).hexdigest()

    xmin = []
    xmax = []
    ymin = []
    ymax = []
    is_crowd = []
    category_names = []
    category_ids = []
    area = []
    encoded_mask_png = []
    num_annotations_skipped = 0
    for object_annotations in annotations_list:
        (x, y, width, height) = tuple(object_annotations['bbox'])
        if width <= 0 or height <= 0:
            num_annotations_skipped += 1
            continue
        if x + width > image_width or y + height > image_height:
            num_annotations_skipped += 1
            continue
        xmin.append(float(x) / image_width)
        xmax.append(float(x + width) / image_width)
        ymin.append(float(y) / image_height)
        ymax.append(float(y + height) / image_height)
        is_crowd.append(object_annotations['iscrowd'])
        category_id = int(object_annotations['category_id'])
        category_ids.append(category_id)
        category_names.append(
            category_index[category_id]['name'].encode('utf8'))
        area.append(object_annotations['area'])

        if include_masks:
            run_len_encoding = mask.frPyObjects(object_annotations['segmentation'],
                                                image_height, image_width)
            binary_mask = mask.decode(run_len_encoding)
            if not object_annotations['iscrowd']:
                binary_mask = np.amax(binary_mask, axis=2)
            pil_image = PIL.Image.fromarray(binary_mask)
            output_io = io.BytesIO()
            pil_image.save(output_io, format='PNG')
            encoded_mask_png.append(output_io.getvalue())
    feature_dict = {
        'image/height':
            dataset_util.int64_feature(image_height),
        'image/width':
            dataset_util.int64_feature(image_width),
        'image/filename':
            dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id':
            dataset_util.bytes_feature(str(image_id).encode('utf8')),
        'image/key/sha256':
            dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded':
            dataset_util.bytes_feature(encoded_jpg),
        'image/format':
            dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin':
            dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax':
            dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin':
            dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax':
            dataset_util.float_list_feature(ymax),
        'image/object/class/text':
            dataset_util.bytes_list_feature(category_names),
        'image/object/is_crowd':
            dataset_util.int64_list_feature(is_crowd),
#         'image/object/area':
#             dataset_util.float_list_feature(area),
    }
    if include_masks:
        feature_dict['image/object/mask'] = (
            dataset_util.bytes_list_feature(encoded_mask_png))
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_dict))
    return key, example, num_annotations_skipped

In [7]:
def create_normal_tf_example(image,
                      image_dir,
                      category_index):
    """Converts image and annotations to a tf.Example proto.

    Args:
      image: dict with keys:
        [u'license', u'file_name', u'coco_url', u'height', u'width',
        u'date_captured', u'flickr_url', u'id']
      image_dir: directory containing the image files.
      category_index: a dict containing COCO category information keyed
        by the 'id' field of each category.  See the
        label_map_util.create_category_index function.
    Returns:
      example: The converted tf.Example
      num_annotations_skipped: Number of (invalid) annotations that were ignored.

    Raises:
      ValueError: if the image pointed to by data['filename'] is not a valid JPEG
    """
    image_height = image['height']
    image_width = image['width']
    filename = image['file_name']
    image_id = image['id']

    full_path = os.path.join(image_dir, filename)
    with tf.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    key = hashlib.sha256(encoded_jpg).hexdigest()

    xmin = []
    xmax = []
    ymin = []
    ymax = []
    is_crowd = []
    category_names = []
    category_ids = []
    area = []
    encoded_mask_png = []
    num_annotations_skipped = 0

    feature_dict = {
        'image/height':
            dataset_util.int64_feature(image_height),
        'image/width':
            dataset_util.int64_feature(image_width),
        'image/filename':
            dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id':
            dataset_util.bytes_feature(str(image_id).encode('utf8')),
        'image/key/sha256':
            dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded':
            dataset_util.bytes_feature(encoded_jpg),
        'image/format':
            dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin':
            dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax':
            dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin':
            dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax':
            dataset_util.float_list_feature(ymax),
        'image/object/class/text':
            dataset_util.bytes_list_feature(category_names),
        'image/object/is_crowd':
            dataset_util.int64_list_feature(is_crowd),
#         'image/object/area':
#             dataset_util.float_list_feature(area),
    }
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_dict))
    return key, example, num_annotations_skipped




In [22]:
def _create_tf_record_from_coco_annotations(
        annotations_file, image_dir, normal_image_dir, output_path, include_masks, num_shards, train):
    """Loads COCO annotation json files and converts to tf.Record format.

    Args:
      annotations_file: JSON file containing bounding box annotations.
      image_dir: Directory containing the image files.
      output_path: Path to output tf.Record file.
      include_masks: Whether to include instance segmentations masks
        (PNG encoded) in the result. default: False.
      num_shards: number of output file shards.
    """
    with contextlib2.ExitStack() as tf_record_close_stack, \
            tf.gfile.GFile(annotations_file, 'r') as fid:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
            tf_record_close_stack, output_path, num_shards)
        groundtruth_data = json.load(fid)
        images = groundtruth_data['images']
        category_index = label_map_util.create_category_index(
            groundtruth_data['categories'])

        annotations_index = {}
        if 'annotations' in groundtruth_data:
            tf.logging.info(
                'Found groundtruth annotations. Building annotations index.')
            for annotation in groundtruth_data['annotations']:
                image_id = annotation['image_id']
                if image_id not in annotations_index:
                    annotations_index[image_id] = []
                annotations_index[image_id].append(annotation)
        missing_annotation_count = 0
        for image in images:
            image_id = image['id']
            if image_id not in annotations_index:
                missing_annotation_count += 1
                annotations_index[image_id] = []
        tf.logging.info('%d images are missing annotations.',
                        missing_annotation_count)

        total_num_annotations_skipped = 0
        
        image_data1 = [(image, 'restricted') for image in images]
        
        
        import os 
        image_data2 = []
        for i,path in enumerate(os.listdir(normal_image_dir)[0:300]):
            image = {}
            img=PIL.Image.open(normal_image_dir+'/'+path)
            width, height = img.size
            image['height'] = height
            image['width'] = width
            image['file_name'] = path
            #large 1459
            image['id'] = i+1460
            image_data2 += [(image,'normal')]
 
        image_data1 = [(image, 'restricted') for image in images]
        
        if train:
            image_data = image_data1[0:1300]
            image_data.extend(image_data2[0:200])

            random.shuffle(image_data)
        else:
            image_data = image_data1[1300:]
            image_data.extend(image_data2[200:])
            random.shuffle(image_data)

        for idx, image in enumerate(image_data):
            if idx % 100 == 0:
                tf.logging.info('On image %d of %d', idx, len(image_data))
            if image[1] == 'restricted':
                #print (image)
                annotations_list = annotations_index[image[0]['id']]
                _, tf_example, num_annotations_skipped = create_tf_example(
                    image[0], annotations_list, image_dir, category_index, include_masks)
                total_num_annotations_skipped += num_annotations_skipped
            else:
                _, tf_example, num_annotations_skipped = create_normal_tf_example(image[0],
                      normal_image_dir,category_index)
                
            shard_idx = idx % num_shards
            output_tfrecords[shard_idx].write(tf_example.SerializeToString())
                
        tf.logging.info('Finished writing, skipped %d annotations.',
                        total_num_annotations_skipped)

In [23]:
train_annotations_file = '/Users/hywel/Downloads/jinnan2_round1_train_20190222/train_no_poly.json' 
train_image_dir = '/Users/hywel/Downloads/jinnan2_round1_train_20190222/restricted' 
normal_image_dir = '/Users/hywel/Downloads/jinnan2_round1_train_20190222/normal' 
output_dir = '/Users/hywel/Downloads/jinnan2_round1_train_20190222'

train_output_path = os.path.join(output_dir, 'jinnan_val.record')

_create_tf_record_from_coco_annotations(
        train_annotations_file,
        train_image_dir,
        normal_image_dir,
        train_output_path,
        False,
        1,
        False)

INFO:tensorflow:Found groundtruth annotations. Building annotations index.
INFO:tensorflow:5 images are missing annotations.
INFO:tensorflow:On image 0 of 260
INFO:tensorflow:On image 100 of 260
INFO:tensorflow:On image 200 of 260
INFO:tensorflow:Finished writing, skipped 0 annotations.


In [ ]:
os.listdir('/Users/hywel/Downloads/jinnan2_round1_train_20190222/normal')[0]

In [ ]:
im = PIL.Image.open('/Users/hywel/Downloads/jinnan2_round1_train_20190222/normal/190105_091930_00150781.jpg')

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
img=Image.open('/Users/hywel/Downloads/jinnan2_round1_train_20190222/normal/190105_091930_00150781.jpg')
plt.imshow(img)
plt.show()

NameError: name 'image' is not defined